<a href="https://colab.research.google.com/github/VanguriBhanusree/My_Project/blob/main/Weather_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # This libraray helps us to fetch data from API

import pandas as pd #for handling and analysing data

import numpy as np #for numerical operations

from sklearn.model_selection import train_test_split #to split data into training and testing sets

import requests # This libraray helps us to fetch data from API

import pandas as pd #for handling and analysing data

import numpy as np #for numerical operations

from sklearn.model_selection import train_test_split #to split data into training and testing sets

from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numericals values

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression ti

import requests # This libraray helps us to fetch data from API

import pandas as pd #for handling and analysing data

import numpy as np #for numerical operations

from sklearn.model_selection import train_test_split #to split data into training and testing sets

from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numericals values

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression

from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions

from datetime import datetime, timedelta

import pandas as pd #for handling and analysing data

import numpy as np #for numerical operations

from sklearn.model_selection import train_test_split #to split data into training and testing sets

from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numericals values

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression ti

from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions

from datetime import datetime, timedelta
import pytz

In [ ]:
API_KEY= '8a9abe6f6904e8a7e2e1f582493efb55'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

1.Fetch Current Data


In [ ]:
def get_curent_weather(city):
  url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response=requests.get(url)
  data=response.json()
  return {
      'city':data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'Wind_Gust_Speed': data['wind']['speed'] #This was the line to edit. Changed wind_gust_speed to Wind_Gust_Speed
  } #Removed the extra lines with unexpected indentation.
      #This ensures that the return statement only includes the intended dictionary

2.Read Historical data

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df=df.dropna()
  df=df.drop_duplicates()
  return df

3. Prepare data for training

In [ ]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define feature variables and target variables
  X = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']] # define X here
  y = data['RainTomorrow']

  return X,y,le # return X instead of x here

4.Train Rain Prediction Model

In [ ]:
def train_rain_model(x,y):
  x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
  model = RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(x_train,y_train)

  y_pred=model.predict(x_test)
  print("MEAN SQUARED ERROR")
  print(mean_squared_error(y_test,y_pred))
  return model

5.Prepare Regression Data

In [ ]:
def prepare_regression_data( data, feature):
  x,y=[],[]
  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])
  x=np.array(x).reshape(-1,1)
  y=np.array(y)
  return x,y

Train Regression Model


In [ ]:
def train_regression_model(x,y):
  model=RandomForestRegressor(n_estimators=100,random_state=42)
  model.fit(x,y)
  return model

Predict Future

In [ ]:
def predict_future(model,current_value):
  predictions=[current_value]
  for i in range(5):
    next_value=model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:]

Weather Analysis function


In [ ]:
 import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

def weather_view():
    city = input('Enter any city name:')
    current_weather = get_curent_weather(city)
    # historical data
    historical_data = read_historical_data('/content/weather.csv')

    # prepare and train the rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360  # Added = for assignment
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]

    # Find compass direction using next and provide a default value (None) if not found
    compass_direction = next((point for point, start, end in compass_points if start <= wind_deg < end), None)

    # Transform compass direction if it's in the LabelEncoder's classes, otherwise assign -1
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
    'MinTemp': current_weather['temp_min'],
    'MaxTemp': current_weather['temp_max'],
    'WindGustDir': compass_direction_encoded,
    'WindGustSpeed': current_weather['Wind_Gust_Speed'],  # Incorrect key used here
    'Humidity': current_weather['humidity'],
    'Pressure': current_weather['pressure'],
    'Temp': current_weather['current_temp']
}


    # Create DataFrame with a single row
    current_df = pd.DataFrame([current_data])

    # rain predict
    rain_prediction = rain_model.predict(current_df)[0]

    # prepare regression model for temperature and humidity
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

    # predict future humidity
    future_humidity = predict_future(hum_model, current_weather['humidity'])
    # predict future temperature
    future_temp = predict_future(temp_model, current_weather['current_temp'])

    # prepare time for future predictions
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f"City: {city} , {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}")
    print(f"Maximum Temperature: {current_weather['temp_max']}")
    print(f"Humidity: {current_weather['humidity']}")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\n Future Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\n Future Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter any city name:Lahore
MEAN SQUARED ERROR
0.1506849315068493
City: Lahore , PK
Current Temperature: 31
Feels Like: 32
Minimum Temperature: 31
Maximum Temperature: 31
Humidity: 48
Weather Prediction: smoke
Rain Prediction: No

 Future Temperature Predictions:
10:00: 31.6°C
11:00: 28.0°C
12:00: 27.8°C
13:00: 25.9°C
14:00: 28.9°C

 Future Humidity Predictions:
10:00: 53.7%
11:00: 58.1%
12:00: 54.2%
13:00: 58.1%
14:00: 54.2%
